In [ ]:
## ODE
import numpy as np
import pandas as pd # for data manipulation
import time
from scipy.integrate import odeint, solve_ivp
from sklearn.metrics import mean_squared_error
import scipy.optimize as optimize

from ipynb.fs.full.myfun_nn import *
from ipynb.fs.full.myfun_model_usefulfuns import *
from ipynb.fs.defs.myfun_plot import *

# NN driven model

In [ ]:
## Traffic dynamic with ANN

def TD_ANN_model(t, X, vel):
    
    """
    Lighthill-Whitham-Richards (LWR) traffic flow model in 1D
    Transform a list as vel into a function of t,X.
    """
    
    d_x = vel
        
    return d_x

## Ode solver for the NN driven model

In [ ]:
def odesolver_ann(x0, vel, t0, tend, deltat = 0.05):
    
    """
    odesolver_ann solves the TD_ANN_model ode system:
    * in [t0, tend] with timestep as deltat,
    * starting from x0
    * vel is the rhs passed to TD_ANN_model to create the model
    """
    
    tspan_ann = time_discretization(t0,tend,deltat)
        
    sol_ann = odeint(TD_ANN_model, x0, tspan_ann, args=(vel,), tfirst = True).T

    return tspan_ann, sol_ann

### Default training

In [ ]:
def create_data_ann_scene(scn):
    
    """
    create_data_ann_scene gives the X and y for an entire scene
    
    X_scn, y_scn = create_data_ann_scene(scn)
    
    X_scn is a list of consecutive distances btw the vehicles of a scene, at each timestamps
    y_scn is a list of approximated velocities, of all the vehicles except the leader one.
    """

    ## Create X
    X_scn = scn['cons_dis']

    ## Create y
    dX_scn = np.diff(scn['Xarr'],axis=1)
    dT_scn = np.diff(scn['Tarr'])
    velocity = dX_scn/dT_scn # velocity at the timestamps

    # we choose the first velocity discretized as (x_(i+1)-x_i)/deltaT
    y_scn = velocity[:-1] #drop the last vehicle
    
    return X_scn, y_scn

In [ ]:
def solve_nn_scn_default(nn_model, scn, epochs, batch_size, v0, deltat=0.05, verbose="auto"):
    
    """
    t_ann_list, x_ann_list, vel_ann_list = odesolver_ann_scene(nn_model, scn, epochs, batch_size, v0, deltat=0.05, verbose="auto")
    
    odesolver_ann_scene solve the ode model driven by a nn in a scene.
    """
 
    x_list, t_list, v_list = [[i] for i in scn['Xarr'][:,0]], [scn['Tarr'][0]], []
    N, tstamps, fmt = scn['N. vehicles'], scn['Tarr'], '{0:.02f}'

    X_arr, y_arr = create_data_ann_scene(scn)

    print("=="*50)
    print(f"We have {len(tstamps)-1} time intervals inside [{fmt.format(tstamps[0])},{fmt.format(tstamps[-1])}]\n")
    
    for i in range(0,len(tstamps)-1):

        print("--"*50)
        print(f"Time interval n.{i}: [{fmt.format(scn['Tarr'][i])}, {fmt.format(scn['Tarr'][i+1])}]\n")
        
        ## STEP 1: Train the NN model and predict the rhs of the TD_ANN_model
        X, y = X_arr[:,i], y_arr[:,i]

        train_nn(nn_model, X, y, epochs, batch_size, verbose)
        y_pred = nn_model(X, training=True).numpy().flatten().tolist()

        ## STEP 2: Solve the ODE sys in this time interval
        x0 = [l[-1] for l in np.vstack(x_list).tolist()]  # last values computed
        t0, tend = scn['Tarr'][i], scn['Tarr'][i+1]
        v_ann = np.append(y_pred,v0).tolist()
        tspan_ann, sol_ann = odesolver_ann(x0, v_ann, t0, tend, deltat=0.05)

        print(f"\
        * y_true: {y}\n\
        * v_ann: {v_ann}\n")
        
        ## STEP 3: store info
        x_list, t_list = update_sol_lists(N, tspan_ann, sol_ann, x_list, t_list)
        v_list.append(v_ann)

        print("--"*50)
    
    print("=="*50)
    
    return t_list, x_list, v_list

### Custom training loop

In [ ]:
def solve_step(model, scn, v0, it, PLOT_ITER, lists, nn_fun):

    t_list, x_list, v_list = lists
    X_arr, y_arr = create_data_ann_scene(scn)
    N, tstamps, fmt = scn['N. vehicles'], scn['Tarr'], '{0:.02f}'

    loss_fn, optimizer = nn_fun
    
    for i in range(0,len(tstamps)-1):
                
        # STEP 1: Create the dataset and train the nn model
        X, y = X_arr[:,i], y_arr[:,i]
        
        ## Train the NN:  Update model coefs
        with tf.GradientTape(persistent=True) as tape:

            # Create tensor that you will watch
            x_tensor = tf.convert_to_tensor(X, dtype=tf.float64)
            tape.watch(x_tensor)

            y_pred = model(X, training=True) # forward pass
            loss_value = loss_fn(y_true = y, y_pred = y_pred) # loss function          

        ## Compute gradients
        trainable_vars = model.trainable_variables
        grads = tape.gradient(loss_value, trainable_vars)
        
        ## Update weights
        optimizer.apply_gradients(zip(grads, trainable_vars))
        
        # STEP 3: Solve the ODE sys in this time interval
        x0 = [l[-1] for l in np.vstack(x_list).tolist()]  # last values computed
        t0, tend = scn['Tarr'][i], scn['Tarr'][i+1]
        v_ann = np.append(y_pred.numpy().flatten().tolist(),v0).tolist()
        tspan_ann, sol_ann = odesolver_ann(x0, v_ann, t0, tend, deltat=0.05)    
        
        # STEP 4: Store the info
        x_list, t_list = update_sol_lists(N, tspan_ann, sol_ann, x_list, t_list)
        v_list.append(v_ann)
                
        if it % PLOT_ITER == 0:
            print(f"\
            - Time interval n.{i}: [{fmt.format(scn['Tarr'][i])}, {fmt.format(scn['Tarr'][i+1])}]\n\
                * y_true: {y}\n\
                * v_ann: {v_ann}\n")
            print("--"*50)
        
    return t_list, x_list, v_list

In [ ]:
def solve_nn_scn_custom(model, scn, v0, LEARNING_RATE_NN = 0.001, LEARNING_RATE_v0=0.5, NUM_ITER=200, PLOT_ITER=25):
    
    """
    t_list, x_list, v_list = solve_nn_scn_custom(model, scn, v0, LEARNING_RATE_NN)
    """
    
    N, tstamps, fmt = scn['N. vehicles'], scn['Tarr'], '{0:.02f}'
    v0_scn = []
    
    print("=="*50)
    print(f"\
    We have {len(tstamps)-1} time intervals inside [{fmt.format(tstamps[0])},{fmt.format(tstamps[-1])}]")

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                    initial_learning_rate=LEARNING_RATE_NN,
                    decay_steps=int(NUM_ITER/2)+1,
                    decay_rate=0.9,
                    staircase=True)
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
    loss_fn = tf.keras.losses.MeanSquaredError()
    
    err_list, err, diff = [], 1e9, 1
    err_list.append(err)
    
    it = 1
    err_best, it_best = err, it
    
    while (diff > 1e-6 and it<NUM_ITER+1):
        
        # STEP 1: Simulate the dynamic over a scene with v0
        x_list, t_list, v_list = [[i] for i in scn['Xarr'][:,0]], [scn['Tarr'][0]], []
        t_list, x_list, v_list = solve_step(model, scn, v0, it, PLOT_ITER,
                                            lists = [t_list, x_list, v_list],
                                            nn_fun = [loss_fn, optimizer])
        _, sol_ann_matched = match_timestamps_scene(t_list, x_list)

        # STEP 2: Update v0 with SGD
        v0_upd, loss_val, grads, g = SGD_v0(scn, sol_ann_matched, v0, LEARNING_RATE_v0) 
        
        ## Store v0 and update it
        v0_scn.append(v0)
        v0 = v0_upd
        
        err = loss_fn(y_true=scn['Xarr'], y_pred = sol_ann_matched).numpy()
        err_list.append(err)

        if err < err_best:
            t_best, x_best, v_best = t_list, x_list, v_list
            it_best, err_best = it, err
        
        #update diff
        if it % 50 == 0:
             diff = abs(err_list[-1]-err_list[-50])
   
        #print(f"it={it}, err={err}")
        it += 1

#     print(f"it best = {it_best}, err = {err_best}")
    print(f"\
    * err= {err}\n\
    * Learning rate NN = {optimizer.learning_rate.numpy()}\n\
    * diff = {diff}")
    
    # plot function
    tscale = 1+(tstamps[-1]-tstamps[0])/10000
    title = f"$df\  n.\ {scn['N. file']}\ -\ Scene\ n.\ {scn.name},\ at\ it={it-1}$"
    plot_scn(scn, sol_ann_matched, title, xbal=0.01, ybal=0.05, scale=tscale)
    
    return t_best, x_best, v_best, v0_scn, it

### Solve the nn-driven model in the all the scenes in a df, and get v0 mean for each scene

In [ ]:
def lr_finder(model, scn, v0):
    
    """
    err_lr_best, lr_best, it_lr_best = lr_finder(model, scn, v0)
    """

    X_arr, y_arr = create_data_ann_scene(scn)
    lr_range = [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001]
    err_lr_best, lr_best = 1e9, None

    for lr in lr_range:

        mmodel = tf.keras.models.clone_model(model)
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
        loss_fn = tf.keras.losses.MeanSquaredError()

        for it in range(25):
            
            x_list, t_list, v_list = [[i] for i in scn['Xarr'][:,0]], [scn['Tarr'][0]], []
            t_list, x_list, v_list = solve_step(mmodel, scn, v0, -1, -2,
                                                lists = [t_list, x_list, v_list],
                                                nn_fun = [loss_fn, optimizer])
            _, sol_ann_matched = match_timestamps_scene(t_list, x_list)            

        err = loss_fn(y_true=scn['Xarr'], y_pred = sol_ann_matched).numpy()  

#         print(f"For lr={lr} we have err={err}")
        if err < err_lr_best:
            err_lr_best, lr_best, it_lr_best = err, lr, it
        
#         print(f"--> err_best={err_lr_best}, lr_best={lr_best}\n")

    return err_lr_best, lr_best, it_lr_best

In [ ]:
def SGD_v0(scn, x_list_matched, v0, LEARNING_RATE_v0):
    
    """
    v0_upd, loss_val, grads, g = SGD_v0(scn, x_list_matched, v0, LEARNING_RATE_v0)
    """
    
    loss_objective = tf.keras.losses.MeanSquaredError()
    
    with tf.GradientTape(persistent=True) as tape:

        trajs_true_tensor = tf.convert_to_tensor(scn['Xarr'], dtype=tf.float64)

        # Create tensor that you will watch
        trajs_pred_tensor = tf.convert_to_tensor(x_list_matched, dtype=tf.float64)
        tape.watch(trajs_pred_tensor)

        loss_val = loss_objective(y_true = trajs_true_tensor, y_pred = trajs_pred_tensor)

    # Compute gradients
    grads = tape.gradient(loss_val, trajs_pred_tensor)

    # Update weights
    g = grads[-1].numpy()[1:].mean() # I'm watching at a mean velocity of the leader car..
    v0_upd = v0 - LEARNING_RATE_v0*g
    
    return v0_upd, loss_val, grads, g

In [ ]:
def solve_nn_df(df, doe, v0, NUM_ITER, LEARNING_RATE_v0=0.5):
    
    """
    Solve nn in a single df ang get info_df, which gives info about v0_scn_mean for each scene.
    
    info_df = solve_nn_df(df, model, v0_guess=30,
                          NUM_EPOCHS=10, LEARNING_RATE_NN=0.01, LEARNING_RATE_v0=0.5, flag_print=False)
    """
    
    scn_list = seq2scn(df)
    info_scn, fmt = [], '{0:.02f}'
    loss_fn = tf.keras.losses.MeanSquaredError()
    model = create_model(doe)
    
    for scn in scn_list:
                
        tstamps = scn['Tarr']
        err_lr_best, lr_best, it_lr_best = lr_finder(model, scn, v0)
       
        print(f"df n.{df['N. file'][0]}, scene n.{scn.name}/{len(scn_list)}")
    
        t_list, x_list, v_list, v0_scn, it = solve_nn_scn_custom(model, scn, v0,
                                                                    lr_best,
                                                                    LEARNING_RATE_v0,
                                                                    NUM_ITER, PLOT_ITER=NUM_ITER)

        # Compute MAE for the solution computed
        _, sol_ann_matched = match_timestamps_scene(t_list, x_list)
        mae = loss_fn(y_true=scn['Xarr'], y_pred = sol_ann_matched).numpy()
        
        v0_scn_mean = np.array(v0_scn).mean()

        info_scn.append([t_list, x_list, v_list, v0_scn, v0_scn_mean, scn.name, df['N. file'][0], it-1])

        print(f"\
        For scene {scn.name}/{len(scn_list)}\n\
        * use LR_NN={lr_best} with err={err_lr_best} at it={it_lr_best}\n\
        * v0_scn_mean = {v0_scn_mean}\n\
        * MAE = {mae}\n")
        print("=="*50)
        print("\n")

    # to better handle, transposte info_df
    tmp = list(map(list, zip(*info_scn)))

    info_df = pd.DataFrame({'t_list': tmp[0],\
                            'x_list': tmp[1],\
                            'v_list': tmp[2],\
                            'v0_scn': tmp[3],\
                            'v0_scn_mean': tmp[4],\
                            'n_scn': tmp[5],\
                            'N. file': tmp[6],\
                            'iter': tmp[7]
                           })
  
    return info_df

### Solve NN driven model in each df of a dataset, and get v0 mean for each scn in each df

In [ ]:
def solve_nn_dataset(doe, v0_guess, dataset, processed_flag, NUM_ITER, LEARNING_RATE_v0=0.5):
    
    """
    info_dataset = solve_nn_dataset(doe, v0_guess, dataset, processed_flag, NUM_ITER, LEARNING_RATE_v0=0.5)
    """
    
    start_time2 = time.time()
    tmp = []
    
    print("--"*50)
    s = '-'.join(str(x) for x in doe)
    print(f"NN structure: {s}")
    
    for step, df in enumerate(dataset):
        
        start_time1 = time.time()
        
        print("**"*50)
        print(f"In df n.{df['N. file'][0]}/{len(dataset)} we have {len(df)} scenes")
        
        info_df = solve_nn_df(df, doe, v0_guess, NUM_ITER, LEARNING_RATE_v0)
        
        # Store info about the NN structure
        nrow = info_df.shape[0]
        info_df['DOE'] = [doe]*nrow
        info_df['processed'] = [processed_flag]*nrow
        
        tmp.append(info_df)

        print(f"For df={df['N. file'][0]} with {len(df)} scenes, time taken:\
        {'{0:.02f}'.format(time.time() - start_time1)}")
        print("**"*50)


    time_taken=time.time() - start_time2
    print(f"\nTime taken for the computation: {'{0:.02f}'.format(time_taken)}")
    print("--"*50)

    # To better handling info_dataset
    info_dataset = pd.concat(tmp, sort=False)
        
    return info_dataset

### Solve NN driven model in the whole dataset, looping over DOE

In [ ]:
# def solve_nn_dataset_DOE(v0_guess, dataset, processed_flag, NUM_ITER, LEARNING_RATE_v0):
    
#     """
#     solve_nn_dataset_DOE(v0_guess, dataset, NUM_ITER, LEARNING_RATE_v0)
#     """
    
#     DOE =[[1,2,1], [1,4,1], [1,10,1]] #Design of experiment
#     tmp = []

#     start_time1 = time.time()
#     for DOE_struct in DOE:

#         print("\n")
#         print("=="*30)

#         s = '-'.join(str(x) for x in DOE_struct)
#         print(f"NN structure: {s}")

#         start_time2 = time.time()

#         info_dataset = solve_nn_dataset(DOE_struct, v0_guess, dataset, NUM_ITER, LEARNING_RATE_v0)

#         # Store info about the NN structure
#         nrow = info_dataset.shape[0]
#         info_dataset['DOE'] = [DOE_struct]*nrow
#         info_dataset['processed'] = [processed_flag]*nrow

#         # Append info_dflist
#         tmp.append(info_dataset)

#         time_taken=time.time() - start_time2
#         print(f"\nTime taken using {s} NN structure: {'{0:.02f}'.format(time_taken)}")
#         print("=="*30)

#     time_taken=time.time() - start_time1
#     print(f"\nTime taken for the computation: {'{0:.02f}'.format(time_taken)}")
    
#     # To better handling info_alldataset
#     info_alldataset = pd.concat(tmp, sort=False);
    
#     return info_alldataset